# IBM Quantum Experience Lab - Lab exercise - Expectation value estimation

This is a group exercise. You will be assigned into groups of 3-4. Please work together through this worksheet using the screen-sharing and video chat facilities in Microsoft Teams or your favourite online environment.

## Background

A promising family of algorithms for near-term quantum computers are called variational optimisation algorithms. 

These are hybrid algorithms with a classical optimiser and a quantum sub-routine. The quantum sub-routines in these algorithms generally have the following form:

1. Make a specified state $|\psi\rangle$.
2. Estimate the expectation value of an observable $A$.

It is the simplicity of this family of quantum circuits which makes it convenient for small scale near-term quantum computers.

In this lab exercise you will compare **two different methods** of estimating an expectation value on the IBMQ chip, and determine which of them performs best.


### Warm up exercise

To get some practise with expectation values, and using QISKIT to estimate them, we warm up with a simple example.

You will estimate the expectation value for a range of states for the observable $Z$.

$Z$ has the matrix form: 

$$ Z = \begin{bmatrix}1 & 0 \\ 0 & -1 \end{bmatrix}$$

Consider a general single qubit state $|{\psi}\rangle = \alpha |0\rangle + \beta |1\rangle$. It has a column vector representation:

$$ |\psi\rangle = \begin{bmatrix}\alpha \\ \beta \end{bmatrix}$$

The expectation value of $Z$ with respect to $|{\psi}\rangle$ is then:

$$ \langle {\psi}| Z|{\psi}\rangle = \begin{bmatrix}\alpha* & \beta* \end{bmatrix}\begin{bmatrix}1 & 0 \\ 0 & -1 \end{bmatrix}\begin{bmatrix}\alpha \\ \beta \end{bmatrix}=|\alpha|^2 - |\beta|^2$$

Although this is an easy calculation, the matrices involved scale in size to $2^n$ by $2^n$ for an $n$-qubit state. Therefore this calculation, in general, is not expected to be efficiently acheivable on a classical computer.

Operator expectation values therefore represent a very simple form of potential quantum speed-up. 

## Estimation on a quantum computer

On a quantum computer, we cannot obtain an expectation value in a single shot. Instead, we need to repeat the experiment many times and calculate the average.

Let's explore this **using the IBMQ simulator**.

Let us initially consider the state $|+\rangle=\sqrt{1/2}(|0\rangle+|1\rangle)$. To measure $Z$ we use the built in standard computational basis measurement. We will need to consider below that the eigenvalues of $Z$ are $+1$ and $-1$ while the outcomes of the measurement in QISKIT are 0 and 1.

The circuit is simple to define:



In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer
from qiskit.tools.visualization import circuit_drawer

In [2]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
circuit = QuantumCircuit(qr, cr)

circuit.h(qr[0])
circuit.measure(qr[0],cr[0])

circuit_drawer(circuit)

┌───┐┌─┐
  q0: ┤ H ├┤M├
      └───┘└╥┘
c0: 1/══════╩═
            0

Let us now simulate this circuit on the simulator:

    

In [3]:
from qiskit import Aer
backend = Aer.get_backend('qasm_simulator')

In [4]:
job = execute(circuit, backend,shots=1024)


In [5]:
result=job.result()
result.get_counts(circuit)

{'1': 501, '0': 523}

To compute the expectation value, we use the following formula:

$$ \langle Z \rangle = \frac{n_0 - n_1}{n_0+n_1}$$

Where $n_0$ represents the number of 0 counts, and $n_1$ represents the number of 1 counts.

**Important**: It is important that you understand why this formula has the form it does.

In [6]:
expectation =(result.get_counts(circuit)['0']-result.get_counts(circuit)['1'])/(result.get_counts(circuit)['0']
                                                                                +result.get_counts(circuit)['1'])

print(expectation)

0.021484375


The accuracy of the estimate of the expectation value will depend upon the number of shots.

**Exercise:** In your Lab Script Notebook, write a short Python routine that uses QISKIT to estimate the expectation value of $Z$ for a range of different number of repetitions (shots) of the measurement. Using Matplotlib to plot your results as a function of the number of shots.

Important: **Use the simulator backend for this exercise**

Things to look for:
* How high does the system allow `shots` to be?
* How does the error in the estimate scale with the number of shots?


# Main lab exercise

The aim of this exercise is to assess the relative performance of the following two methods for expectation value estimation when implemented on IBM's chip.

## Family of observables

The observables we will consider are the following family:

$$ M(\theta) = \cos (\theta) X + \sin (\theta) Y  = \begin{bmatrix} 0 & \cos (\theta) - i \sin (\theta)  \\ \cos (\theta) + i \sin (\theta) & 0\end{bmatrix} = \begin{bmatrix} 0 & \exp[-i \theta] \\ \exp[i \theta]  & 0\end{bmatrix}
$$

Recall the "hardware defined gate", $u_1(\lambda)$:

$$ u_1(\lambda)=\begin{bmatrix}1 & 0\\0 & \exp(i \lambda)\end{bmatrix}$$

We can write $M(\theta)$ in terms of $ u_1(\lambda)$ and $X$ as:

$$M(\theta) = u_1(\theta) X u_1(-\theta)$$


## Family of states

The states we will consider here are the following set:

$$ |{\psi (\phi)}\rangle=\cos(\phi) |0\rangle + i\sin(\phi)|1\rangle= e^{-i \phi}H u_1(-2\phi) H|{0}\rangle $$








## Method 1 (unitary basis rotation method)

We estimate $\langle M \rangle$ where $M$ shares the same spectrum as $Z$, by finding the unitary $U$ such that

$$ M = U Z U^\dagger $$

We can then effectively measure $$M$$ by:

* Applying gates to realise $$U^\dagger$$.
* Measuring $$Z$$.

By repeating such measurements and analysising the outcomes, we may estimate $\langle M \rangle$


## Method 2 (standard VQE method)

We identify $\alpha$, $\beta$, $\gamma$ and $\delta$ such that

$$M = \alpha X + \beta Y + \gamma Z + \delta I$$ 

and effectively estimate $$\langle M \rangle $$ by:

* Estimating (as many of) $\langle X \rangle $,  $\langle Y \rangle $,  $\langle Z \rangle $ (as is necessary).
* Using the formula:

$$ \langle M \rangle = \alpha \langle X \rangle + \beta \langle Y \rangle + \gamma \langle Z \rangle + \delta \langle I \rangle $$ 






## Aim of the Lab Exercise:

Working as a group, estimate the expectation value for the pairs of $\theta$, $\phi$ specified below, using the two methods above on IBM's chip, calculating the errors obtained with respect to the exact value, and analysing which method performs best (and why) in these cases.

Pair 1: $\theta= \pi/4$, $\phi= 0$
Pair 2: $\theta= \pi/8$, $\phi= \pi/4$

To do so, use QISKIT to implement code that implements each of the two methods, and computes expectation value estimates for these pairs. Compare the estimate to the precise value, computed via matrix multiplication.

### Tips and advice:


* Write and test your code using the simulator first.
* Be aware that access to the physical chip may be withdrawn at any time (though IBMQ now have many chips available and down-time is usually short).
* Use the chips with the shortest queues for your experiments. 



## Assessment

You will be assessed as a group for this exercise. 

Please submit the following via Moodle (only one person in the group needs to submit it):

* A Jupyter Notebook containing the code used to generated your notes.

Please annotate your code, so it is clearly explained.

* A 1-2 page report summarising your results.

The report should contain, a brief introduction, a description of your methods (referring to the Python notebook as appropriate) and a summary and discussion of your findings.

Deadline:  **4pm Monday 21st Feb 2022**. 

**Please ensure that both the Notebook and the Report contain the names of all members of your group**.

A marking guideline can found found on Moodle.

